---
layout: code-post
title: Recurrent neural networks
tags: [neural nets]
---

By now a classicial and outdated technique, I am going to try to implement a
recurrent neural network and see how well it fits some dummy data.
A good resource for an overview of RNNs is [this cheat sheet](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks)
from a Stanford CS course.

Outline:
1. Architecture
2. Backpropagation through time (BTT)
3. Dummy training examples
4. LSTMs and other cell types

In [146]:
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt

## 1. Architecture

RNNs are based on _cells_ and act on sequences of inputs. A single (simple) cell will take
two inputs: one element of the input sequence and either the output or the
_hidden state_ of the previous cell in the sequence. One can also take the two
inputs to be the output of the previous cell along with the hidden state of the 
previous cell if there is no longer an input sequence to rely upon. The many
variations can be found in the cheat sheet referenced above. A cell can also
take more than two inputs, as in the context of Long Short-Term Memory cells
there is an additional _context_ that exists in addition to the hidden state
that is passed between cells.

Let $x\_t$ be the $t$th element of the input sequence and $h\_{t-1}$ the
hidden state from the $(t-1)$st cell. Then the new hidden state is given
by
$$
    h\_t = \sigma(W\_{xh}x\_t + W\_{hh}h\_{t-1} + b\_h)
$$
where $\sigma$ is the activation function (we will take this to be ReLU).
If inpute sequence element $x\_t$ has dimension $n\_x$ and the hidden
state has size $n\_h$, then $W\_{xh}$ has size $n\_h\times n\_x$
while $W\_{hh}$ is a square matrix of size $n\_h\times n\_h$. This is equivalent
to
$$
    h\_t = \sigma([W\_{xh} | W\_{hh}](x\_t\oplus h\_{t-1} + b\_h)
$$
where$\oplus$ indicates vector concatenation and $[W\_{xh} | W\_{hh}]$ is a matrix
of size $n\_h \times(n\_x + n\_h)$. Many of the descriptions of RNNs, and in particular
when cells become more complicated as in _Long Short-Term Memory_ cells, formulate
things in terms of concatenation, so I just wanted to quickly show that these were
equivalent.

Moving forward, I will say that the hidden state is given by
$$
    h\_t = \sigma(W\_h(x\_t\oplus h\_{t-1}) + b\_h)
$$
where $W\_h$ has size $n\_h\times(n\_x + n\_h)$. Without working it out, my
intuition -- which could be wrong! -- suggests that this means we can keep
formulating the initialization in terms of the input and output vector sizes.

The output is then given by
$$
    y\_t = \sigma(W\_yh\_t + b\_y).
$$

All of these words are why you will usually just see diagrams when discussing
varieties of LSTMs.

The simplified version of an RNN would actually just take the hidden state and
the output to be the same, with no extra layer between the hidden
state and the output.

In the following code we implement a `Cell` class that initializes based on
input size, hidden state size, and output size. It has methods to take an
input and update the hidden state and update both the hidden state and its
output, which it stores. The updates can also return the inputs to the hidden
state neuron layer and the output neuron layers that are fed into the
ReLU and sigmoid activations, respectively.

In [148]:
def relu(x):
    return x * (x>0) + 0 * (x<=0)

def relu_prime(x):
    return 1 * (x>0) + 0 * (x<=0)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

class Cell():
    
    def __init__(self, input_size, hidden_size, output_size, random_seed=None):                 
        """ give the size of the input vectors, hidden state
        vectors, and output sizes. 
        
        If using the cells in such a way that the output is
        used in place of the hidden state, then hidden size and
        output size must be the same.
        
        If using in such a way that the output is used in place
        of the input at some point, then the output and input
        sizes must be the same.
        """
        np.random.seed(random_seed)
        
        self._input_size = input_size
        self._hidden_size = hidden_size
        self._output_size = output_size
        
        self._hidden_state = np.zeros((self._hidden_size, 1))
        self._y = np.zeros((self._output_size, 1))
        
        def initialize(nrows, ncols, init_type=None):
            # Kaiming He initialiation for weights
            # or just some arbitrary initialization for biases
            if ncols > 1:
                if init_type=='He':
                    std = np.sqrt(2.0 / ncols)
                elif init_type=='Xavier':
                    std = np.sqrt(2.0 / (ncols + nrows))
                else:
                    msg = 'init_type must be He or Xavier if initializing weights'
                    raise Exception(msg)
            else:
                # this is for biases
                std = np.sqrt(2.0 / nrows)
            
            try_again = True
            while try_again:
                w = np.random.normal(0, std, size=(nrows, ncols))
                try_again = abs(np.std(w)-std) > .05 or abs(np.mean(w)) > .05
                
            return w
        
        self._wh = initialize(self._hidden_size, self._input_size + self._hidden_size, 'He')
        self._bh = initialize(self._hidden_size, 1)
        self._wy = initialize(self._output_size, self._hidden_size, 'Xavier')
        self._by = initialize(self._output_size, 1)
        
    @property
    def input_size(self):
        return self._input_size
    
    @property
    def hidden_size(self):
        return self._hidden_size
    
    @property
    def output_size(self):
        return self._output_size
    
    @property
    def hidden_state(self):
        return self._hidden_state
    
    def _shape_assert(self, oldval, newval):
        assert oldval.shape == newval.shape, \
            "new value must have shape {}".format(oldval.shape)
    
    @hidden_state.setter
    def hidden_state(self, val):
        self._shape_assert(self.hidden_state, val)
        self._hidden_state = val
        
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self, val):
        self._shape_assert(self.y, val)
        self._y = val
        
    @property
    def wh(self):
        return self._wh
    
    @wh.setter
    def wh(self, val):
        self._shape_assert(self.wh, val)
        self._wh = val
        
    @property
    def bh(self):
        return self._bh
    
    @bh.setter
    def bh(self, val):
        self._shape_assert(self.bh, val)
        self._bh = val
        
    @property
    def wy(self):
        return self._wy
    
    @wy.setter
    def wy(self, val):
        self._shape_assert(self.wy, val)
        self._wy = val
        
    @property
    def by(self):
        return self._by
    
    @by.setter
    def by(self, val):
        self._shape_assert(self.by, val)
        self._by = val
        
    def update_hidden_state(self, x, return_activation_input=False):
        """ take a numpy vector of size (input_size, 1) and
        calculate the hidden state of size (hidden_size, 1).
        
        This returns nothing, updating in place. """
        
        assert type(x) == np.ndarray, "x must by numpy.ndarray"
        assert x.shape[0] == self.input_size, \
            "x must have first dim of length {}".format(self.input_size)
        assert x.shape[1] == 1, "x must have second dim of length 1"
        
        # concatenate
        xh = np.concatenate([x, self.hidden_state])
        
        # get output
        activation_input = np.matmul(self.wh, xh) + self.bh
        self.hidden_state = relu(activation_input)
        
        if return_activation_input:
            return activation_input
        else:
            return None
        
    def update_y(self, return_activation_input=False):
        """ takes the hidden state and runs through a neuron
        layer with sigmoid activation function to get the output y """
        activation_input = np.matmul(self.wy, self.hidden_state) + self.by
        self.y = sigmoid(activation_input)
        
        if return_activation_input:
            return activation_input
        else:
            return None
        
    def update(self, x, return_activation_inputs=False):
        hidden_layer_input = self.update_hidden_state(x, return_activation_inputs)
        output_layer_input = self.update_y(return_activation_inputs)
        
        if return_activation_inputs:
            return hidden_layer_input, output_layer_input
        else:
            return None
        
        

Thinking about unfolding the RNN in time, we have a single layer of cells.
It also possible to have multiple cells at a single timestep. At timestep
$t$ the first cell takes the intput $x\_t$ and the hidden state $h\_{t-1, 1}$
from the previous timestep of the same cells. The second cell would take
the output $y\_{t, 1}$ and its previous hidden state $h\_{t-1, 2}$ as its
input and put out the output $y\_t$. Unrolling this structure in time would
lead to multiple layers of cells. It seems that only one or two layers of cells
are typical, as training is very expensive for RNNs.